In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [33]:
# зчитуємо дані з таблиці sqlite для прогнозування
engine = create_engine('sqlite:///ts.sqlite3', echo=False)
sqlite_connection = engine.connect()

from sqlalchemy import inspect
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

# Get column information
print(inspector.get_columns('ts1'))

df = pd.read_sql_query("SELECT * from ts1", engine)
df

sqlite_connection.close()

['ts1']
[{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'time', 'type': DATETIME(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


In [34]:
# в таблиці sqlite дані дату у нас в текстовому форматі, тож переформатовуємо в datetime
df['time'] = pd.to_datetime(df['time'])

In [35]:
# видаляємо зайву колонку index
df.drop(columns='index',inplace = True)



,time
0,2015-10-15
1,2015-10-07
2,2015-08-20
3,2015-03-31
4,2015-03-05
...,...
7440,2016-08-04
7441,2018-01-02
7442,2017-02-01
7443,2016-08-11


In [5]:

from warnings import simplefilter

import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    mean_absolute_percentage_error,
)
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series
from sktime.forecasting.bats import BATS
from sktime.forecasting.tbats import TBATS

simplefilter("ignore", FutureWarning)
%matplotlib inline

In [18]:
# зрізаємо нерелеватний період
#df = df[df.time.dt.year>2015]

In [36]:
# перетворюємо дату у формат періоду та підраховуємо кількість подій у цьому періоді
ts1 = df.resample('M',on='time').count()

In [37]:
# першій та останній період скоріш за все будуть неповними, можливо, це буде впливати на прогноз,можливо,ні, але на графіку
# буде виглядати обрубано, тому їх видаляємо
ts1 = ts1[:-1]
ts1 = ts1[1:]


In [38]:
# resample ставить дату в індекс, але переформатовує дату на об'єкт,переводимо індекс знову в формат періоду
# та зберігаємо в зміній y
ts1.index = ts1.index.to_period()
y = ts1.time
# для горизонта передбачення краще використати щось типу max_date минус декілька періодів, а не абсолютну дату, ДОРОБИТИ
max_date = max(ts1.index).strftime('%Y-%m-%d')


In [48]:
fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range('2018-09', periods=12, freq="M")), is_relative=False
 )

In [49]:

def prediction_table(y,forecaster,fh,method):
    forecaster.fit(y)
    y_pred = forecaster.predict(fh)
    y_er = y['2018-09-01':]
    max_y = max(y.index).strftime('%Y-%m-%d')
    pred_err = y_pred['2018-09-01':max_y]
    y_pred.index = y_pred.index.to_timestamp()
    pred = pd.DataFrame(y_pred).reset_index()
    pred['method']=method
    pred['MAPE']=mean_absolute_percentage_error(y_er, pred_err)
    pred.columns=[['date','value','method','MAPE']]
    return pred
    
#mean_absolute_percentage_error(y_pred, y)

In [42]:
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)

In [44]:
forecaster.fit(y)

ExponentialSmoothing(seasonal='additive', sp=12, trend='add')

In [50]:
pred1 = prediction_table(y,forecaster,fh,'ExponentialSmoothing')

In [51]:
forecaster = AutoARIMA(sp=12, suppress_warnings=True)
pred2 = prediction_table(y,forecaster,fh,'AutoARIMA')

C:\Users\olga\anaconda3\lib\site-packages\pmdarima\arima\arima.py:577: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "


In [52]:
forecaster = BATS(sp=12, use_trend=True, use_box_cox=True)
pred3 = prediction_table(y,forecaster,fh,'BATS')

In [53]:
forecaster = EnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add",  seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add",  seasonal="multiplicative", sp=12))
])
pred4 = prediction_table(y,forecaster,fh,'EnsembleForecaster')

In [54]:
forecaster = TBATS(sp=12, use_trend=True, use_box_cox=True)
pred5 = prediction_table(y,forecaster,fh,'TBATS')

In [55]:
z = y.copy()
z = z.to_timestamp(freq="M")
fhz = ForecastingHorizon(
    pd.date_range('2018-09', periods=12, freq="M"), is_relative=False
 )

In [56]:
from sktime.forecasting.fbprophet import Prophet
forecaster = Prophet(
    seasonality_mode="multiplicative",
    n_changepoints=int(len(y) / 8),
    add_country_holidays={"country_name": "Ukraine"},
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
)
forecaster.fit(z)
y_pred = forecaster.predict(fhz)

In [59]:
pr_prophet = pd.DataFrame(y_pred).reset_index()
pr_prophet['ds'] = pd.to_datetime(pr_prophet['ds'])
pr_prophet['method']='Prophet'
z_er = z['2018-09':]
max_z = max(z.index).strftime('%Y-%m-%d')
pred_err = y_pred['2018-09':max_z]
pr_prophet['MAPE']=mean_absolute_percentage_error(z_er, pred_err)
pr_prophet.columns=[['date','value','method','MAPE']]
pr_prophet

,date,value,method,MAPE
0,2018-09-30,210.946563,Prophet,0.06639
1,2018-10-31,212.506937,Prophet,0.06639
2,2018-11-30,215.336974,Prophet,0.06639
3,2018-12-31,200.107658,Prophet,0.06639
4,2019-01-31,195.805244,Prophet,0.06639
5,2019-02-28,167.052748,Prophet,0.06639
6,2019-03-31,179.260638,Prophet,0.06639
7,2019-04-30,229.272319,Prophet,0.06639
8,2019-05-31,286.023388,Prophet,0.06639
9,2019-06-30,225.717892,Prophet,0.06639


In [61]:
#y.index = y.index.to_timestamp()

fact = pd.DataFrame(y)
fact.rename(columns={'time':'val'},inplace=True)

fact.reset_index(inplace=True)

fact['method']='fact'

fact.columns=[['date','value','method']]

full = pd.concat([fact,pred1,pred2,pred3,pred4,pred5,pr_prophet])

full

,MAPE,date,method,value
0,NaN,2015-01-01,fact,11.000000
1,NaN,2015-02-01,fact,27.000000
2,NaN,2015-03-01,fact,41.000000
3,NaN,2015-04-01,fact,40.000000
4,NaN,2015-05-01,fact,49.000000
...,...,...,...,...
7,0.06639,2019-04-30,Prophet,229.272319
8,0.06639,2019-05-31,Prophet,286.023388
9,0.06639,2019-06-30,Prophet,225.717892
10,0.06639,2019-07-31,Prophet,272.976885


In [62]:
# запис у таблицю
sqlite_connection = engine.connect()
sqlite_table = "ts_pred"
full.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
sqlite_connection.close()